In [1]:
import os
import numpy as np
import json
import neuroglancer
from matplotlib import pyplot as plt

viewer = neuroglancer.Viewer()
viewer

http://127.0.0.1:39661/v/14862a732af9fb27b874c5b9071d6ab7671fe5f2/

In [2]:
color_filepath = os.path.join('../', 'neuroglancer/contours/json_cache', 'struct_to_color_2.json')
with open(color_filepath, 'r') as json_file:
    colors = json.load(json_file)
colors = {name.upper(): index for name, index in colors.items()}
    
surround = False
VOL_DIR = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes'
files = os.listdir(VOL_DIR)
volume_files = sorted([f for f in files if f.endswith('.npy') and surround == ('surround' in f)])
origin_files = sorted([f for f in files if f.endswith('.txt') and surround == ('surround' in f)])
    
structure_volume_origin = {}
for volume_filename, origin_filename in zip(volume_files, origin_files):
    prefix = os.path.splitext(volume_filename)[0]
    structure = prefix.replace('atlasV7_10.0um_scoreVolume_', '').replace('_surround_200um', '')
    if structure not in origin_filename:
        print(structure, origin_filename)
        break
    
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure.upper())
        color = colors[sided]

    volume = np.load(os.path.join(VOL_DIR, volume_filename))
    origin = np.loadtxt(os.path.join(VOL_DIR, origin_filename))
    
    volume = np.rot90(volume, axes=(0,1))
    volume = np.flip(volume, axis=0)
    volume[volume > 0.8] = color
    volume = volume.astype(np.uint8)
    
    structure_volume_origin[structure] = (volume, origin)
print(structure_volume_origin.keys())

dict_keys(['10N_L', '10N_R', '12N', '3N_L', '3N_R', '4N_L', '4N_R', '5N_L', '5N_R', '6N_L', '6N_R', '7N_L', '7N_R', '7n_L', '7n_R', 'AP', 'Amb_L', 'Amb_R', 'DC_L', 'DC_R', 'IC', 'LC_L', 'LC_R', 'LRt_L', 'LRt_R', 'PBG_L', 'PBG_R', 'Pn_L', 'Pn_R', 'RMC_L', 'RMC_R', 'RtTg', 'SC', 'SNC_L', 'SNC_R', 'SNR_L', 'SNR_R', 'Sp5C_L', 'Sp5C_R', 'Sp5I_L', 'Sp5I_R', 'Sp5O_L', 'Sp5O_R', 'Tz_L', 'Tz_R', 'VCA_L', 'VCA_R', 'VCP_L', 'VCP_R', 'VLL_L', 'VLL_R'])


In [3]:
x_length = 1000
y_length = 1000
z_length = 300
full_brain_volume_annotated = np.zeros((x_length, y_length, z_length), dtype=np.uint8)

for structure, (volume, origin) in structure_volume_origin.items():  
    
    x, y, z = origin
    x_start = int(x) + x_length // 2
    y_start = int(y) + y_length // 2
    z_start = int(z) // 2 + z_length // 2
    x_end = x_start + volume.shape[0]
    y_end = y_start + volume.shape[1]
    z_end = z_start + (volume.shape[2] + 1) // 2

    z_indices = [z for z in range(volume.shape[2]) if z % 2 == 0]
    volume = volume[:, :, z_indices]
    full_brain_volume_annotated[x_start:x_end, y_start:y_end,z_start:z_end] += volume


all_volume_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 20]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['all'] = all_volume_layer

In [5]:
volume_name = 'annotations.npy'
with open(volume_name, 'wb') as file:
    np.save(file, full_brain_volume_annotated)

In [32]:
from cloudvolume import CloudVolume

info = CloudVolume.create_new_info(
    num_channels    = 1,
    layer_type      = 'segmentation',
    data_type       = 'uint8', # Channel images might be 'uint8'
    encoding        = 'raw', # raw, jpeg, compressed_segmentation, fpzip, kempressed
    resolution      = [10000, 10000, 20000], # Voxel scaling, units are in nanometers
    voxel_offset    = [0, 0, 0], # x,y,z offset in voxels from the origin
    mesh            = 'mesh',
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size      = [ 512, 512, 16 ], # units are voxels
    volume_size     = full_brain_volume_annotated.shape, # e.g. a cubic millimeter dataset
)
vol = CloudVolume('file:///home/sherry/Desktop/test', info=info, mip=0)
vol.add_scale((2, 2, 2))

{'encoding': 'raw',
 'chunk_sizes': [[64, 64, 16]],
 'resolution': [20000, 20000, 40000],
 'voxel_offset': [0, 0, 0],
 'size': [500, 500, 150],
 'key': '20000_20000_40000'}

In [37]:
vol.bbox_to_mip(full_brain_volume_annotated.shape, 0, 1)

Bbox([500, 500, 150],[501, 501, 151], dtype=int32)

In [34]:
help(vol.bbox_to_mip)

Help on method bbox_to_mip in module cloudvolume.frontends.precomputed:

bbox_to_mip(bbox, mip, to_mip) method of cloudvolume.frontends.precomputed.CloudVolumePrecomputed instance
    Convert bbox or slices from one mip level to another.



In [38]:
vol.commit_info()
vol[:, :, :] = full_brain_volume_annotated[:, :, :]

Uploading: 100%|██████████| 76/76 [00:03<00:00, 23.67it/s]
